In [1]:
import os
import json
import random
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import open3d as o3d
import torch
from torch import nn

from plyfile import PlyData, PlyElement
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import cdist

from glob import glob
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### 1. Prepare PCL Data

In [2]:
class PointCloudModel(nn.Module):
    def __init__(self, max_sh_degree=3, device="cpu"):
        super(PointCloudModel, self).__init__()
        self.max_sh_degree = max_sh_degree
        self._xyz = None
        self._features_dc = None
        self._features_rest = None
        self._opacity = None
        self._scaling = None
        self._rotation = None
        self.pretrained_exposures = None
        self.device = device

    def load_ply(self, path, use_train_test_exp=False):
        """
        Additional Explanation of the point cloud model's features:
        |---------------------------------------------------------------------------------------------------------------------------------------------------|
        | Variable              | Meaning                                                 | Typical Shape (if RGB)                                          |
        | --------------------- | ------------------------------------------------------- | --------------------------------------------------------------- |
        | `self._features_dc`   | Zeroth-order SH coefficients (constant per point color) | `[N_points, 3]`                                                 |
        | `self._features_rest` | Higher-order SH coefficients (view-dependent features)  | `[N_points, 3 x (n_SH - 1)]` where `n_SH = (max_sh_degree+1)^2` |
        |---------------------------------------------------------------------------------------------------------------------------------------------------|
        """
        plydata = PlyData.read(path)
        if use_train_test_exp:
            exposure_file = os.path.join(os.path.dirname(path), os.pardir, os.pardir, "exposure.json")
            if os.path.exists(exposure_file):
                with open(exposure_file, "r") as f:
                    exposures = json.load(f)
                self.pretrained_exposures = {image_name: torch.FloatTensor(exposures[image_name]).requires_grad_(False).cuda() for image_name in exposures}
                print(f"Pretrained exposures loaded.")
            else:
                print(f"No exposure to be loaded at {exposure_file}")
                self.pretrained_exposures = None

        xyz = np.stack((np.asarray(plydata.elements[0]["x"]),
                        np.asarray(plydata.elements[0]["y"]),
                        np.asarray(plydata.elements[0]["z"])),  axis=1)
        opacities = np.asarray(plydata.elements[0]["opacity"])[..., np.newaxis]

        features_dc = np.zeros((xyz.shape[0], 3, 1))
        features_dc[:, 0, 0] = np.asarray(plydata.elements[0]["f_dc_0"])
        features_dc[:, 1, 0] = np.asarray(plydata.elements[0]["f_dc_1"])
        features_dc[:, 2, 0] = np.asarray(plydata.elements[0]["f_dc_2"])

        # extra_f_names = [p.name for p in plydata.elements[0].properties if p.name.startswith("f_rest_")]
        # extra_f_names = sorted(extra_f_names, key = lambda x: int(x.split('_')[-1]))
        # assert len(extra_f_names)==3*(self.max_sh_degree + 1) ** 2 - 3
        # features_extra = np.zeros((xyz.shape[0], len(extra_f_names)))
        # for idx, attr_name in enumerate(extra_f_names):
        #     features_extra[:, idx] = np.asarray(plydata.elements[0][attr_name])
        # # Reshape (P,F*SH_coeffs) to (P, F, SH_coeffs except DC)
        # features_extra = features_extra.reshape((features_extra.shape[0], 3, (self.max_sh_degree + 1) ** 2 - 1))

        scale_names = [p.name for p in plydata.elements[0].properties if p.name.startswith("scale_")]
        scale_names = sorted(scale_names, key = lambda x: int(x.split('_')[-1]))
        scales = np.zeros((xyz.shape[0], len(scale_names)))
        for idx, attr_name in enumerate(scale_names):
            scales[:, idx] = np.asarray(plydata.elements[0][attr_name])

        rot_names = [p.name for p in plydata.elements[0].properties if p.name.startswith("rot")]
        rot_names = sorted(rot_names, key = lambda x: int(x.split('_')[-1]))
        rots = np.zeros((xyz.shape[0], len(rot_names)))
        for idx, attr_name in enumerate(rot_names):
            rots[:, idx] = np.asarray(plydata.elements[0][attr_name])

        self._xyz = nn.Parameter(torch.tensor(xyz, dtype=torch.float, device=self.device).requires_grad_(True))
        self._features_dc = nn.Parameter(torch.tensor(features_dc, dtype=torch.float, device=self.device).transpose(1, 2).contiguous().requires_grad_(True))
        # self._features_rest = nn.Parameter(torch.tensor(features_extra, dtype=torch.float, device=self.device).transpose(1, 2).contiguous().requires_grad_(True))
        self._opacity = nn.Parameter(torch.tensor(opacities, dtype=torch.float, device=self.device).requires_grad_(True))
        self._scaling = nn.Parameter(torch.tensor(scales, dtype=torch.float, device=self.device).requires_grad_(True))
        self._rotation = nn.Parameter(torch.tensor(rots, dtype=torch.float, device=self.device).requires_grad_(True))

        self.active_sh_degree = self.max_sh_degree


    def convert_ply_to_df(self, beta, alpha, id, df_range=[], add_spatial_features=False, do_log=False):
        
        data_dict = {
            'position': self._xyz.detach().numpy(),
            'f_dc': self._features_dc.detach().numpy(),
            # 'f_rest': self._features_rest.detach().numpy(),
            'opacity': self._opacity.detach().numpy(),
            'scale': self._scaling.detach().numpy(),
            'rotation': self._rotation.detach().numpy(),
            'beta': beta * np.ones(self._opacity.detach().numpy().shape),
            'alpha': alpha * np.ones(self._opacity.detach().numpy().shape),
            'is_fog': (beta!=0) * np.ones(self._opacity.detach().numpy().shape),
            'id': id * np.ones(self._opacity.detach().numpy().shape)
        }

        
        # Flatten each into separate columns
        df_dict = {}

        # Flatten position
        for i, axis in enumerate(['x', 'y', 'z']):
            df_dict[f'pos_{axis}'] = data_dict['position'][:, i]

        # Flatten feature_dc
        for i, color in enumerate(['0', '1', '2']):
            df_dict[f'f_dc_{color}'] = data_dict['f_dc'][:, 0, i]

        # Flatten scale
        for i, axis in enumerate(['x', 'y', 'z']):
            df_dict[f'scale_{axis}'] = data_dict['scale'][:, i]

        # Flatten rotation
        for i, axis in enumerate(['0', '1', '2', '3']):
            df_dict[f'rot_{axis}'] = data_dict['rotation'][:, i]

        # Opacity
        df_dict['opacity'] = data_dict['opacity'].squeeze()

        # Flatten feature_rest
        df_dict['beta'] = data_dict['beta'].squeeze()
        df_dict['alpha'] = data_dict['alpha'].squeeze()
        df_dict['is_fog'] = data_dict['is_fog'].squeeze()
        df_dict['id'] = data_dict['id'].squeeze()

        
        df = pd.DataFrame(df_dict) if df_range == [] else pd.DataFrame(data_dict).iloc[df_range[0]:df_range[1]]

        if add_spatial_features:
            return self.add_spatial_features(df, do_log=do_log)
        else:
            return df


    def add_spatial_features(self, df, output_file_path=None, radius_list=[0.2, 0.6], k_neighbors=[6, 12], do_log=False):
        """
        Add surrounding gaussian information to each point in the dataset.
        
        Parameters:
        df: pandas.DataFrame - Input dataframe with gaussian data
        output_file_path: str - Path to save the enhanced dataset (optional)
        radius_list: list - List of radii to search for neighbors
        k_neighbors: list - List of k values for k-nearest neighbors
        """
        
        # Load the dataset or use provided dataframe
        if isinstance(df, str):
            if do_log: print("Loading dataset...")
            df = pd.read_csv(df)
        else:
            if do_log: print("Using provided dataframe...")
        if do_log: print(f"Dataset shape: {df.shape}")
        
        # Extract 3D positions
        positions = df[['pos_x', 'pos_y', 'pos_z']].values
        
        # Initialize new feature columns
        new_features = {}
        
        if do_log: print("Computing spatial features...")
        
        # 1. K-Nearest Neighbors features
        if do_log: print("Computing k-nearest neighbors features...")
        for k in k_neighbors:
            if do_log: print(f"  Processing k={k}...")
            
            # Fit k-NN model
            nbrs = NearestNeighbors(n_neighbors=k+1, algorithm='ball_tree').fit(positions)
            distances, indices = nbrs.kneighbors(positions)
            
            # Remove self (first neighbor is always the point itself)
            distances = distances[:, 1:]
            indices = indices[:, 1:]
            
            # Add features based on k-nearest neighbors
            new_features[f'knn_{k}_mean_distance'] = np.mean(distances, axis=1)
            new_features[f'knn_{k}_std_distance'] = np.std(distances, axis=1)
            new_features[f'knn_{k}_max_distance'] = np.max(distances, axis=1)
            new_features[f'knn_{k}_min_distance'] = np.min(distances, axis=1)
            
            # Density-like features
            new_features[f'knn_{k}_density'] = k / (np.mean(distances, axis=1) ** 3)  # Points per unit volume
            
            # Neighbor properties aggregation
            neighbor_opacity = df.iloc[indices.flatten()]['opacity'].values.reshape(indices.shape)
            
            new_features[f'knn_{k}_mean_opacity'] = np.mean(neighbor_opacity, axis=1)
            new_features[f'knn_{k}_std_opacity'] = np.std(neighbor_opacity, axis=1)
            
            # Scale and rotation variation in neighborhood
            for scale_col in ['scale_x', 'scale_y', 'scale_z']:
                neighbor_scale = df.iloc[indices.flatten()][scale_col].values.reshape(indices.shape)
                new_features[f'knn_{k}_{scale_col}_std'] = np.std(neighbor_scale, axis=1)
        
        # 2. Radius-based features
        if do_log: print("Computing radius-based features...")
        for radius in radius_list:
            if do_log: print(f"  Processing radius={radius}...")
            
            # Find neighbors within radius
            nbrs_radius = NearestNeighbors(radius=radius, algorithm='ball_tree').fit(positions)
            distances, indices = nbrs_radius.radius_neighbors(positions)
            
            # Count neighbors within radius (excluding self)
            neighbor_counts = np.array([len(dist) - 1 for dist in distances])  # -1 to exclude self
            new_features[f'radius_{radius}_count'] = neighbor_counts
            
            # Local density
            new_features[f'radius_{radius}_density'] = neighbor_counts / (4/3 * np.pi * radius**3)
            
            # Average properties of neighbors within radius
            radius_mean_opacity = []
            radius_mean_distance = []
            
            for i, (dists, idxs) in enumerate(zip(distances, indices)):
                if len(idxs) > 1:  # More than just self
                    # Remove self (distance 0)
                    mask = dists > 0
                    neighbor_idxs = idxs[mask]
                    neighbor_dists = dists[mask]
                    
                    if len(neighbor_idxs) > 0:
                        radius_mean_opacity.append(df.iloc[neighbor_idxs]['opacity'].mean())
                        radius_mean_distance.append(np.mean(neighbor_dists))
                    else:
                        radius_mean_opacity.append(0)
                        radius_mean_distance.append(radius)
                else:
                    radius_mean_opacity.append(0)
                    radius_mean_distance.append(radius)
            
            new_features[f'radius_{radius}_mean_opacity'] = radius_mean_opacity
            new_features[f'radius_{radius}_mean_distance'] = radius_mean_distance
        
        # # 3. Global position features
        # if do_log: print("Computing global position features...")
        
        # # Distance from center of mass
        # center_of_mass = np.mean(positions, axis=0)
        # distances_from_center = np.linalg.norm(positions - center_of_mass, axis=1)
        # new_features['distance_from_center'] = distances_from_center
        
        # # Height-based features (assuming z is up)
        # new_features['height_percentile'] = df['pos_z'].rank(pct=True) * 100
        
        # 4. Local geometric features
        if do_log: print("Computing local geometric features...")
        
        # For each point, compute the covariance of its nearest neighbors to understand local structure
        k_for_geometry = 10
        nbrs_geom = NearestNeighbors(n_neighbors=k_for_geometry+1, algorithm='ball_tree').fit(positions)
        _, indices_geom = nbrs_geom.kneighbors(positions)
        indices_geom = indices_geom[:, 1:]  # Remove self
        
        local_planarity = []
        local_sphericity = []
        local_linearity = []
        
        for i, neighbor_idx in enumerate(indices_geom):
            neighbor_points = positions[neighbor_idx]
            # Center the points
            centered_points = neighbor_points - np.mean(neighbor_points, axis=0)
            
            # Compute covariance matrix
            if len(centered_points) > 3:  # Need at least 4 points for 3D covariance
                try:
                    cov_matrix = np.cov(centered_points.T)
                    eigenvalues = np.linalg.eigvals(cov_matrix)
                    eigenvalues = np.sort(eigenvalues)[::-1]  # Sort in descending order
                    eigenvalues = np.maximum(eigenvalues, 1e-10)  # Avoid division by zero
                    
                    # Geometric features based on eigenvalues
                    linearity = (eigenvalues[0] - eigenvalues[1]) / eigenvalues[0]
                    planarity = (eigenvalues[1] - eigenvalues[2]) / eigenvalues[0]
                    sphericity = eigenvalues[2] / eigenvalues[0]
                except:
                    linearity = planarity = sphericity = 0
            else:
                linearity = planarity = sphericity = 0
                
            local_linearity.append(linearity)
            local_planarity.append(planarity)
            local_sphericity.append(sphericity)
        
        new_features['local_linearity'] = local_linearity
        new_features['local_planarity'] = local_planarity
        new_features['local_sphericity'] = local_sphericity
        
        # 5. Scale and rotation context features
        if do_log: print("Computing scale and rotation context features...")
        
        # Compare current point's scale with neighborhood average
        k_context = 10
        nbrs_context = NearestNeighbors(n_neighbors=k_context+1, algorithm='ball_tree').fit(positions)
        _, indices_context = nbrs_context.kneighbors(positions)
        indices_context = indices_context[:, 1:]  # Remove self
        
        for scale_col in ['scale_x', 'scale_y', 'scale_z']:
            scale_ratios = []
            for i, neighbor_idx in enumerate(indices_context):
                current_scale = df.iloc[i][scale_col]
                neighbor_scales = df.iloc[neighbor_idx][scale_col].values
                mean_neighbor_scale = np.mean(neighbor_scales)
                if mean_neighbor_scale > 1e-10:  # Avoid division by very small numbers
                    ratio = current_scale / mean_neighbor_scale
                else:
                    ratio = 1.0
                scale_ratios.append(ratio)
            new_features[f'{scale_col}_neighborhood_ratio'] = scale_ratios
        
        # Add all new features to the dataframe
        if do_log: print("Adding features to dataframe...")
        for feature_name, feature_values in new_features.items():
            df[feature_name] = feature_values
        
        if do_log: print(f"Added {len(new_features)} new features")
        if do_log: print(f"Enhanced dataset shape: {df.shape}")
        
        # Save the enhanced dataset
        if output_file_path:
            df.to_csv(output_file_path, index=False)
            if do_log: print(f"Enhanced dataset saved to: {output_file_path}")
        
        # Print feature summary
        if do_log: print("\nNew features added:")
        for feature in sorted(new_features.keys()):
            if do_log: print(f"  - {feature}")
        
        return df

    def convert_df_to_ply(self, df, output_path, normals=None):
        # means, scales, rotations, rgbs, opacities, normals=None):
        means = df[['pos_x', 'pos_y', 'pos_z']].values
        scales = df[['scale_x', 'scale_y', 'scale_z']].values
        rotations = df[['rot_0', 'rot_1', 'rot_2', 'rot_3']].values
        colors = df[['f_dc_0', 'f_dc_1', 'f_dc_2']].values
        opacities = df['opacity'].values[..., np.newaxis]

        if normals is None:
            normals = np.zeros_like(means)

        if scales.shape[1] == 1:
            scales = np.tile(scales, (1, 3))

        attrs = ['x', 'y', 'z',
                'nx', 'ny', 'nz',
                'f_dc_0', 'f_dc_1', 'f_dc_2',
                'opacity',
                'scale_0', 'scale_1', 'scale_2',
                'rot_0', 'rot_1', 'rot_2', 'rot_3',]

        dtype_full = [(attribute, 'f4') for attribute in attrs]
        elements = np.empty(means.shape[0], dtype=dtype_full)

        attributes = np.concatenate((means, normals, colors, opacities, scales, rotations), axis=1)
        elements[:] = list(map(tuple, attributes))
        el = PlyElement.describe(elements, 'vertex')
        PlyData([el]).write(output_path)

        print(f"Saved PLY format Splat to {output_path}")


### 2.1. Dataset1: KPro 

Since "density" has been replaced with "beta" and "alpha," the following program is currently not executable.

In [48]:
PLY_PATH1 = "/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/not_fog/pcl_kpro_outside_50000.ply"
PLY_PATHs = glob("/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/fog/pcl_kpro_outside_density*.ply")


# Load the point cloud
pcm1 = PointCloudModel(max_sh_degree=3)
pcm1.load_ply(PLY_PATH1, use_train_test_exp=False)

pcms = [pcm1]
for i, PLY_PATH in enumerate(PLY_PATHs):
    print(f"Loading {PLY_PATH}...")
    pcm = PointCloudModel(max_sh_degree=3)
    pcm.load_ply(PLY_PATH, use_train_test_exp=False)
    pcms.append(pcm)

len(pcms)

Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/fog/pcl_kpro_outside_density070_50000.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/fog/pcl_kpro_outside_density025_50000.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/fog/pcl_kpro_outside_density040_50000.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/fog/pcl_kpro_outside_density005_50000.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/fog/pcl_kpro_outside_density060_50000.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/fog/pcl_kpro_outside_density015_50000.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/fog/pcl_kpro_outside_density050_50000.ply...


8

In [49]:
df0 = pcms[0].convert_ply_to_df(density=0, id=1)[-1]
df2 = pcms[1].convert_ply_to_df(density=0.05, id=2)[-1]
df3 = pcms[2].convert_ply_to_df(density=0.15, id=3)[-1]
df4 = pcms[3].convert_ply_to_df(density=0.25, id=4)[-1]
df5 = pcms[4].convert_ply_to_df(density=0.40, id=5)[-1]
df6 = pcms[5].convert_ply_to_df(density=0.50, id=6)[-1]
df7 = pcms[6].convert_ply_to_df(density=0.60, id=7)[-1]
df8 = pcms[7].convert_ply_to_df(density=0.70, id=8)[-1]

Using provided dataframe...
Dataset shape: (50000, 17)
Computing spatial features...
Computing k-nearest neighbors features...
  Processing k=6...
  Processing k=12...
Computing radius-based features...
  Processing radius=0.2...
  Processing radius=0.6...
Computing global position features...
Computing local geometric features...
Computing scale and rotation context features...
Adding features to dataframe...
Added 44 new features
Enhanced dataset shape: (50000, 61)

New features added:
  - distance_from_center
  - height_percentile
  - knn_12_density
  - knn_12_fog_ratio
  - knn_12_max_distance
  - knn_12_mean_density
  - knn_12_mean_distance
  - knn_12_mean_opacity
  - knn_12_min_distance
  - knn_12_scale_x_std
  - knn_12_scale_y_std
  - knn_12_scale_z_std
  - knn_12_std_distance
  - knn_12_std_opacity
  - knn_6_density
  - knn_6_fog_ratio
  - knn_6_max_distance
  - knn_6_mean_density
  - knn_6_mean_distance
  - knn_6_mean_opacity
  - knn_6_min_distance
  - knn_6_scale_x_std
  - knn

In [50]:
train_df = pd.concat([df0, df2, df3, df4, df5, df6, df7, df8], ignore_index=True)

train_df

,pos_x,pos_y,pos_z,f_dc_0,f_dc_1,f_dc_2,scale_x,scale_y,scale_z,rot_0,...,radius_0.6_mean_density,radius_0.6_mean_distance,distance_from_center,height_percentile,local_linearity,local_planarity,local_sphericity,scale_x_neighborhood_ratio,scale_y_neighborhood_ratio,scale_z_neighborhood_ratio
0,-1.861856,-3.231780,-2.033842,-1.550611,-0.543492,0.160501,-5.152133,-2.677765,-2.478002,0.883647,...,0.0,0.447112,4.173511,0.308000,0.467021,0.427652,0.105327,1.0,1.0,1.0
1,-1.833929,-3.065194,-1.719714,-0.932350,-0.289042,0.992111,-1.420711,-1.370266,-1.393824,0.392667,...,0.0,0.454056,3.946898,23.572000,0.198583,0.375091,0.426326,1.0,1.0,1.0
2,-1.763869,-2.796995,-1.733285,0.924700,1.967847,2.427304,-2.561936,-2.400023,-4.045022,0.455044,...,0.0,0.449400,3.698304,22.676000,0.214136,0.415843,0.370021,1.0,1.0,1.0
3,-1.860865,-3.189933,-1.072952,0.148762,-0.065396,0.384061,-2.004849,-0.292106,-10.770756,0.294250,...,0.0,0.514279,4.000659,54.466000,0.459477,0.456776,0.083747,1.0,1.0,1.0
4,-2.046119,-2.805880,-1.341710,-1.178835,0.681085,2.195142,-1.155659,-2.774600,-0.904581,1.205504,...,0.0,0.477036,3.616953,44.156000,0.323731,0.504793,0.171475,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58923,0.910801,0.346761,9.804516,0.976406,1.006499,0.460732,-1.916141,-2.226333,-2.488414,0.706442,...,0.7,0.410243,10.014898,99.424460,0.996977,0.002473,0.000550,1.0,1.0,1.0
58924,1.510251,0.569756,10.062398,1.356460,1.441350,1.531047,-1.322731,-1.350691,-2.864459,0.462506,...,0.7,0.322572,10.337642,99.568345,0.992612,0.006086,0.001302,1.0,1.0,1.0
58925,1.313448,0.541304,10.116633,0.132603,0.180467,0.286846,-1.527483,-1.691768,-2.129508,0.805801,...,0.7,0.373438,10.367670,99.712230,0.996830,0.001906,0.001264,1.0,1.0,1.0
58926,1.302707,0.547599,10.129763,0.117627,0.167459,0.271103,-1.442894,-1.599403,-2.906422,0.505370,...,0.7,0.377328,10.379894,99.856115,0.996831,0.001912,0.001256,1.0,1.0,1.0


### 2.2. Dataset2: Tandt

In [3]:
DATASET_NAME = "train"   # 'truck' or 'train'

In [4]:
FOG_PLY_PATHs   = sorted(glob(f"/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/{DATASET_NAME}/fog_gaussian/{DATASET_NAME}_*.ply"))
OTHER_PLY_PATHs = sorted(glob(f"/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/{DATASET_NAME}/other_gaussian/{DATASET_NAME}_*.ply"))[1:]

# Load the point cloud
def load_ply_files(PLY_PATHs):
    pcms = {}

    for PLY_PATH in PLY_PATHs:
        print(f"Loading {PLY_PATH}...")
        key = PLY_PATH.split(f'/{DATASET_NAME}_')[-1].split('.')[0]
        pcm = PointCloudModel(max_sh_degree=3)
        pcm.load_ply(PLY_PATH, use_train_test_exp=False)
        pcms[key] = pcm
    
    pcms = dict(
        sorted(
            pcms.items(),
            key=lambda kv: (
                int(kv[0].split('_')[0].replace('beta', '')),
                int(kv[0].split('_')[1].replace('alpha', ''))
            )
        )
    )
    
    return pcms

fog_pcms, other_pcms = load_ply_files(FOG_PLY_PATHs), load_ply_files(OTHER_PLY_PATHs)

len(fog_pcms), len(other_pcms)

Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/train/fog_gaussian/train_beta1_alpha025.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/train/fog_gaussian/train_beta1_alpha050.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/train/fog_gaussian/train_beta1_alpha075.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/train/fog_gaussian/train_beta1_alpha100.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/train/fog_gaussian/train_beta1_alpha125.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/train/fog_gaussian/train_beta1_alpha150.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/train/fog_gaussian/train_beta1_alpha175.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/train/fog_gaussian/train_beta1_

(60, 60)

In [6]:
def summarize_pcms(pcms_dicts, labels=("fog", "other")):
    for label, pcms in zip(labels, pcms_dicts):
        print(f"\n{label.upper()} PLY files:")
        for param, pcm in pcms.items():
            # support "beta3_alpha050" or "truck_beta3_alpha050"
            parts = param.split("_")
            beta = int(parts[-2].replace("beta", ""))
            alpha = int(parts[-1].replace("alpha", ""))
            
            df = pcm.convert_ply_to_df(beta=beta, alpha=alpha, id=-1, do_log=False)
            print(f"  {param:<25} {len(df):>6} points")

# call once for both sets
summarize_pcms([fog_pcms, other_pcms])



FOG PLY files:
  beta1_alpha025              1110 points
  beta1_alpha050              1679 points
  beta1_alpha075              1740 points
  beta1_alpha100              1767 points
  beta1_alpha125              1934 points
  beta1_alpha150              2051 points
  beta1_alpha175              2413 points
  beta1_alpha200              2412 points
  beta1_alpha225              3048 points
  beta1_alpha250              3174 points
  beta2_alpha025              1295 points
  beta2_alpha050              1109 points
  beta2_alpha075               998 points
  beta2_alpha100              1136 points
  beta2_alpha125              1219 points
  beta2_alpha150              1470 points
  beta2_alpha175              1878 points
  beta2_alpha200              2138 points
  beta2_alpha225              2415 points
  beta2_alpha250              2824 points
  beta3_alpha025              1242 points
  beta3_alpha050              1135 points
  beta3_alpha075               970 points
  beta3_alpha100  

#### a. Save dataFrame together (fog & others)

##### a-1. Original features

In [13]:
# Scene with only non-foggy point clouds
id = 0
fog_param = "beta0_alpha000"

PLY_PATH = f"/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/{DATASET_NAME}/other_gaussian/{DATASET_NAME}_beta0_alpha000.ply"
pcm = PointCloudModel(max_sh_degree=3)
pcm.load_ply(PLY_PATH, use_train_test_exp=False)
df = pcm.convert_ply_to_df(beta=0, alpha=0, id=id)

df.to_csv(f"/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/csv/train/{DATASET_NAME}/{DATASET_NAME}_scene_{fog_param}.csv", index=False)

In [14]:
# Scene with only foggy point clouds
id = 1

dfs = []
for fog_param, fog_pcm in fog_pcms.items():

    beta_str, alpha_str = fog_param.split("_")
    beta = int(beta_str.replace("beta", ""))
    alpha = int(alpha_str.replace("alpha", ""))

    df = fog_pcm.convert_ply_to_df(beta=beta, alpha=alpha, id=id)
    df.to_csv(f"/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/csv/train/{DATASET_NAME}/{DATASET_NAME}_fog_{fog_param}.csv", index=False)

    dfs.append(df)

    id += 1

df = pd.concat(dfs, ignore_index=True)
df.to_csv(f"/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/csv/train/{DATASET_NAME}/{DATASET_NAME}_fog_all.csv", index=False)

In [15]:
# Scene with both fog and non-foggy point clouds
id = 1
for (fog_param, fog_pcm), (other_param, other_pcm) in zip(fog_pcms.items(), other_pcms.items()):

    assert fog_param == other_param, f"Mismatch in parameters: {fog_param} vs {other_param}"

    beta_str, alpha_str = fog_param.split("_")
    beta = int(beta_str.replace("beta", ""))
    alpha = int(alpha_str.replace("alpha", ""))

    dfs = []
    print(f"Processing dataset {DATASET_NAME}_scene_{fog_param} with beta={beta}, alpha={alpha}...")

    dfs.append(fog_pcm.convert_ply_to_df(beta=beta, alpha=alpha, id=id))
    dfs.append(other_pcm.convert_ply_to_df(beta=0, alpha=0, id=id))

    train_df = pd.concat(dfs, ignore_index=True)
    train_df.to_csv(f"/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/csv/train/{DATASET_NAME}/{DATASET_NAME}_scene_{fog_param}.csv", index=False)

    id += 1

Processing dataset truck_scene_beta1_alpha025 with beta=1, alpha=25...
Processing dataset truck_scene_beta1_alpha050 with beta=1, alpha=50...
Processing dataset truck_scene_beta1_alpha075 with beta=1, alpha=75...
Processing dataset truck_scene_beta1_alpha100 with beta=1, alpha=100...
Processing dataset truck_scene_beta1_alpha125 with beta=1, alpha=125...
Processing dataset truck_scene_beta1_alpha150 with beta=1, alpha=150...
Processing dataset truck_scene_beta1_alpha175 with beta=1, alpha=175...
Processing dataset truck_scene_beta1_alpha200 with beta=1, alpha=200...
Processing dataset truck_scene_beta1_alpha225 with beta=1, alpha=225...
Processing dataset truck_scene_beta1_alpha250 with beta=1, alpha=250...
Processing dataset truck_scene_beta2_alpha025 with beta=2, alpha=25...
Processing dataset truck_scene_beta2_alpha050 with beta=2, alpha=50...
Processing dataset truck_scene_beta2_alpha075 with beta=2, alpha=75...
Processing dataset truck_scene_beta2_alpha100 with beta=2, alpha=100..

##### a-2. Original with spatial features

In [13]:
def add_spatial_features_subdataset(df, dataset_size=200_000, slider_step=100_000, radius_list=[0.5], k_neighbors=[10], do_log=False):

    dfs = []
    for start_idx in range(0, len(df), slider_step):
        end_idx = min(start_idx + dataset_size, len(df))
        if do_log: print(f"Processing sub-dataset with indices [{start_idx}:{end_idx}]...")
        sub_df = df[start_idx:end_idx]
        sub_df = pcm.add_spatial_features(sub_df, radius_list=radius_list, k_neighbors=k_neighbors, do_log=do_log)
        dfs.append(sub_df)
        if end_idx == len(df):
            break
    
    df = pd.concat(dfs, ignore_index=True)

    return df


In [13]:
# Scene with only non-foggy point clouds
id = 0
fog_param = "beta0_alpha000"

PLY_PATH = f"/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/{DATASET_NAME}/other_gaussian/{DATASET_NAME}_beta0_alpha000.ply"
pcm = PointCloudModel(max_sh_degree=3)
pcm.load_ply(PLY_PATH, use_train_test_exp=False)
df = pcm.convert_ply_to_df(beta=0, alpha=0, id=id)
df = add_spatial_features_subdataset(df, do_log=True)

df.to_csv(f"/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/csv/train/dataset_spatial/{DATASET_NAME}/{DATASET_NAME}_scene_{fog_param}.csv", index=False)

: 

In [14]:
# Scene with only foggy point clouds
id = 1

dfs = []
pcm = PointCloudModel(max_sh_degree=3)
for fog_param, fog_pcm in tqdm(fog_pcms.items()):

    beta_str, alpha_str = fog_param.split("_")
    beta = int(beta_str.replace("beta", ""))
    alpha = int(alpha_str.replace("alpha", ""))

    df = fog_pcm.convert_ply_to_df(beta=beta, alpha=alpha, id=id)
    df = add_spatial_features_subdataset(df)
    df.to_csv(f"/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/csv/train/dataset_spatial/{DATASET_NAME}/{DATASET_NAME}_fog_{fog_param}.csv", index=False)

    dfs.append(df)

    id += 1

df = pd.concat(dfs, ignore_index=True)
df = add_spatial_features_subdataset(df)
df.to_csv(f"/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/csv/train/dataset_spatial/{DATASET_NAME}/{DATASET_NAME}_fog_all.csv", index=False)

100%|██████████| 60/60 [00:24<00:00,  2.41it/s]


In [ ]:
# Scene with both fog and non-foggy point clouds
id = 1
for (fog_param, fog_pcm), (other_param, other_pcm) in tqdm(zip(fog_pcms.items(), other_pcms.items())):

    assert fog_param == other_param, f"Mismatch in parameters: {fog_param} vs {other_param}"

    beta_str, alpha_str = fog_param.split("_")
    beta = int(beta_str.replace("beta", ""))
    alpha = int(alpha_str.replace("alpha", ""))

    dfs = []
    print(f"Processing dataset {DATASET_NAME}_scene_{fog_param} with beta={beta}, alpha={alpha}...")

    dfs.append(fog_pcm.convert_ply_to_df(beta=beta, alpha=alpha, id=id))
    dfs.append(other_pcm.convert_ply_to_df(beta=0, alpha=0, id=id))

    df = pd.concat(dfs, ignore_index=True)
    df = add_spatial_features_subdataset(df)
    df.to_csv(f"/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/csv/train/dataset_spatial/{DATASET_NAME}/{DATASET_NAME}_scene_{fog_param}.csv", index=False)

    id += 1

### 3. Test Dataset

In [42]:
CSV_PATH = "/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/csv/train/train/train_fog_all.csv"
df = pd.read_csv(CSV_PATH)

pcm.convert_df_to_ply(df, "/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/output/temp/train_fog_all.ply", normals=None)

Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/output/temp/train_fog_all.ply


In [ ]:
PLY_PATH = "/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/playroom/playroom_beta2_alpha225.ply"

pcm = PointCloudModel(max_sh_degree=3)
pcm.load_ply(PLY_PATH, use_train_test_exp=False)
test_df = pcm.convert_ply_to_df(beta=1, alpha=0, id=1, df_max_size=350000)[-1]

test_df

Using provided dataframe...
Dataset shape: (350000, 18)
Computing spatial features...
Computing k-nearest neighbors features...
  Processing k=6...
  Processing k=12...
Computing radius-based features...
  Processing radius=0.2...
  Processing radius=0.6...
Computing global position features...
Computing local geometric features...
Computing scale and rotation context features...
Adding features to dataframe...
Added 36 new features
Enhanced dataset shape: (350000, 54)

New features added:
  - distance_from_center
  - height_percentile
  - knn_12_density
  - knn_12_max_distance
  - knn_12_mean_distance
  - knn_12_mean_opacity
  - knn_12_min_distance
  - knn_12_scale_x_std
  - knn_12_scale_y_std
  - knn_12_scale_z_std
  - knn_12_std_distance
  - knn_12_std_opacity
  - knn_6_density
  - knn_6_max_distance
  - knn_6_mean_distance
  - knn_6_mean_opacity
  - knn_6_min_distance
  - knn_6_scale_x_std
  - knn_6_scale_y_std
  - knn_6_scale_z_std
  - knn_6_std_distance
  - knn_6_std_opacity
  - 

,pos_x,pos_y,pos_z,f_dc_0,f_dc_1,f_dc_2,scale_x,scale_y,scale_z,rot_0,...,radius_0.6_mean_opacity,radius_0.6_mean_distance,distance_from_center,height_percentile,local_linearity,local_planarity,local_sphericity,scale_x_neighborhood_ratio,scale_y_neighborhood_ratio,scale_z_neighborhood_ratio
0,-6.445139,-3.891315,-4.713235,0.534943,-0.208325,-1.438138,-0.593268,-0.261250,-1.938807,0.761683,...,4.190296,0.404139,8.130576,21.023714,0.746523,0.159741,0.093736,1.0,1.0,1.0
1,-6.223934,-3.639636,-4.740130,0.332738,-0.403247,-1.642925,-1.700558,-3.151525,-0.426919,0.818274,...,5.716631,0.387899,7.878646,19.980857,0.692361,0.220746,0.086893,1.0,1.0,1.0
2,-6.490288,-4.125004,-4.284445,1.679884,1.575285,1.938681,-3.124252,-0.285324,-1.937053,0.940454,...,8.488198,0.450180,8.049953,22.622857,0.725976,0.189992,0.084032,1.0,1.0,1.0
3,-6.585819,-3.866310,-4.344097,4.329207,1.821168,0.217360,-3.342154,-2.533765,-0.832650,0.779118,...,4.548640,0.412019,7.995402,22.534000,0.284710,0.630055,0.085236,1.0,1.0,1.0
4,-6.424289,-3.594277,-4.591394,-1.531458,-1.179737,-1.110180,-4.849475,-3.093928,-1.410571,1.099618,...,5.275579,0.312089,7.887877,22.046857,0.528341,0.374668,0.096991,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349995,1.803958,3.619145,1.667952,0.406461,-1.783658,-1.787741,-3.080219,-2.449448,-4.140055,0.993476,...,-1.096436,0.474370,4.834887,60.998000,0.478601,0.146870,0.374529,1.0,1.0,1.0
349996,1.913437,3.588664,1.609249,-0.100334,-1.774341,-1.336415,-2.639458,-4.198310,-4.799247,1.236711,...,-1.556344,0.379034,4.872757,60.618000,0.466451,0.225187,0.308362,1.0,1.0,1.0
349997,1.852911,3.504643,1.762249,0.913960,-1.774341,-1.352922,-2.557059,-2.740345,-5.123179,1.194673,...,-1.442064,0.464918,4.830508,61.721143,0.456663,0.280150,0.263187,1.0,1.0,1.0
349998,1.876868,3.607365,1.674611,0.430741,-1.774341,-0.829064,-2.909540,-4.628372,-5.350671,0.788926,...,-1.469515,0.432563,4.880448,61.054571,0.280132,0.523683,0.196185,1.0,1.0,1.0


### Applendix: Codes for making datasets with spatial features

In [16]:
DATASET_NAMEs = ['truck', 'train']


for DATASET_NAME in DATASET_NAMEs:
    
    FOG_PLY_PATHs   = sorted(glob(f"/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/{DATASET_NAME}/fog_gaussian/{DATASET_NAME}_*.ply"))
    OTHER_PLY_PATHs = sorted(glob(f"/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/{DATASET_NAME}/other_gaussian/{DATASET_NAME}_*.ply"))[1:]


    fog_pcms, other_pcms = load_ply_files(FOG_PLY_PATHs), load_ply_files(OTHER_PLY_PATHs)

    # Scene with only non-foggy point clouds
    id = 0
    fog_param = "beta0_alpha000"

    PLY_PATH = f"/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/{DATASET_NAME}/other_gaussian/{DATASET_NAME}_beta0_alpha000.ply"
    pcm = PointCloudModel(max_sh_degree=3)
    pcm.load_ply(PLY_PATH, use_train_test_exp=False)
    df = pcm.convert_ply_to_df(beta=0, alpha=0, id=id)
    df = add_spatial_features_subdataset(df)

    df.to_csv(f"/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/csv/train/dataset_spatial/{DATASET_NAME}/{DATASET_NAME}_scene_{fog_param}.csv", index=False)


    # Scene with only foggy point clouds
    id = 1

    dfs = []
    pcm = PointCloudModel(max_sh_degree=3)
    for fog_param, fog_pcm in tqdm(fog_pcms.items()):

        beta_str, alpha_str = fog_param.split("_")
        beta = int(beta_str.replace("beta", ""))
        alpha = int(alpha_str.replace("alpha", ""))

        df = fog_pcm.convert_ply_to_df(beta=beta, alpha=alpha, id=id)
        df = add_spatial_features_subdataset(df)
        df.to_csv(f"/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/csv/train/dataset_spatial/{DATASET_NAME}/{DATASET_NAME}_fog_{fog_param}.csv", index=False)

        dfs.append(df)

        id += 1

    df = pd.concat(dfs, ignore_index=True)
    df = add_spatial_features_subdataset(df)
    df.to_csv(f"/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/csv/train/dataset_spatial/{DATASET_NAME}/{DATASET_NAME}_fog_all.csv", index=False)

    # Scene with both fog and non-foggy point clouds
    id = 1
    for (fog_param, fog_pcm), (other_param, other_pcm) in tqdm(zip(fog_pcms.items(), other_pcms.items())):

        assert fog_param == other_param, f"Mismatch in parameters: {fog_param} vs {other_param}"

        beta_str, alpha_str = fog_param.split("_")
        beta = int(beta_str.replace("beta", ""))
        alpha = int(alpha_str.replace("alpha", ""))

        dfs = []
        print(f"Processing dataset {DATASET_NAME}_scene_{fog_param} with beta={beta}, alpha={alpha}...")

        dfs.append(fog_pcm.convert_ply_to_df(beta=beta, alpha=alpha, id=id))
        dfs.append(other_pcm.convert_ply_to_df(beta=0, alpha=0, id=id))

        df = pd.concat(dfs, ignore_index=True)
        df = add_spatial_features_subdataset(df)
        df.to_csv(f"/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/csv/train/dataset_spatial/{DATASET_NAME}/{DATASET_NAME}_scene_{fog_param}.csv", index=False)

        id += 1

Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/fog_gaussian/truck_beta1_alpha025.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/fog_gaussian/truck_beta1_alpha050.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/fog_gaussian/truck_beta1_alpha075.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/fog_gaussian/truck_beta1_alpha100.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/fog_gaussian/truck_beta1_alpha125.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/fog_gaussian/truck_beta1_alpha150.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/fog_gaussian/truck_beta1_alpha175.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/fog_gaussian/truck_beta1_

100%|██████████| 60/60 [00:37<00:00,  1.61it/s]
0it [00:00, ?it/s]

Processing dataset truck_scene_beta1_alpha025 with beta=1, alpha=25...


1it [08:09, 489.27s/it]

Processing dataset truck_scene_beta1_alpha050 with beta=1, alpha=50...


...

58it [3:50:11, 50.54s/it]

Processing dataset train_scene_beta6_alpha225 with beta=6, alpha=225...


59it [3:50:56, 48.83s/it]

Processing dataset train_scene_beta6_alpha250 with beta=6, alpha=250...


60it [3:51:38, 231.64s/it]


In [ ]:
df = pd.read_csv("/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/csv/train/dataset_original/train/train_fog_all.csv")

df.columns

Index(['pos_x', 'pos_y', 'pos_z', 'f_dc_0', 'f_dc_1', 'f_dc_2', 'scale_x',
       'scale_y', 'scale_z', 'rot_0', 'rot_1', 'rot_2', 'rot_3', 'opacity',
       'beta', 'alpha', 'is_fog', 'id'],
      dtype='object')

: 